# Установка библиотеки RDKit

In [ ]:
pip -q install rdkit-pypi

# Подключение библиотек

In [ ]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import warnings

from rdkit import Chem
from rdkit import RDLogger
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.Draw import MolsToGridImage

warnings.filterwarnings("ignore")
RDLogger.DisableLog("rdApp.*")

np.random.seed(29)
tf.random.set_seed(29)

# Загрузка данных

In [ ]:
csv_path = keras.utils.get_file(
    "Lipophilicity.csv", "https://deepchemdata.s3-us-west-1.amazonaws.com/datasets/Lipophilicity.csv"
)

### Извлечение признаков

In [ ]:
 class FeatureExtraction:
    def __init__(self, allowable_sets):
        self.dim = 0
        self.features_mapping = {}
        for k, s in allowable_sets.items():
            s = sorted(list(s))
            self.features_mapping[k] = dict(zip(s, range(self.dim, len(s) + self.dim)))
            self.dim += len(s)

    def encode(self, inputs):
        output = np.zeros((self.dim,))
        for name_feature, feature_mapping in self.features_mapping.items():
            feature = getattr(self, name_feature)(inputs)
            if feature not in feature_mapping:
                continue
            output[feature_mapping[feature]] = 1.0
        return output


class AtomFeatureExtraction(FeatureExtraction):
    def __init__(self, allowable_sets):
        super().__init__(allowable_sets)

    def symbol(self, atom):
        return atom.GetSymbol()

    def n_valence(self, atom):
        return atom.GetTotalValence()

    def n_hydrogens(self, atom):
        return atom.GetTotalNumHs()

    def hybridization(self, atom):
        return atom.GetHybridization().name.lower()


class BondFeatureExtraction(FeatureExtraction):
    def __init__(self, allowable_sets):
        super().__init__(allowable_sets)
        self.dim += 1

    def encode(self, bond):
        output = np.zeros((self.dim,))
        if bond is None:
            output[-1] = 1.0
            return output
        output = super().encode(bond)
        return output

    def bond_type(self, bond):
        return bond.GetBondType().name.lower()

    def conjugated(self, bond):
        return bond.GetIsConjugated()


atom_extracter = AtomFeatureExtraction(
    allowable_sets={
        "symbol": {"B", "Br", "C", "Ca", "Cl", "F", "H", "I", "N", "Na", "O", "P", "S"},
        "n_valence": {0, 1, 2, 3, 4, 5, 6},
        "n_hydrogens": {0, 1, 2, 3, 4},
        "hybridization": {"s", "sp", "sp2", "sp3"},
    }
)

bond_extracter = BondFeatureExtraction(
    allowable_sets={
        "bond_type": {"single", "double", "triple", "aromatic"},
        "conjugated": {True, False},
    }
)


### Создание трёхмерного тензора для описания молекулярного графа

In [ ]:

def molecule_creator(smiles):
    molecule = Chem.MolFromSmiles(smiles, sanitize=False)

    flag = Chem.SanitizeMol(molecule, catchErrors=True)
    if flag != Chem.SanitizeFlags.SANITIZE_NONE:
        Chem.SanitizeMol(molecule, sanitizeOps=Chem.SanitizeFlags.SANITIZE_ALL ^ flag)

    Chem.AssignStereochemistry(molecule, cleanIt=True, force=True)
    return molecule


def graph_creator(molecule):

    atom_features = []
    bond_features = []
    pair_indices = []

    for atom in molecule.GetAtoms():
        atom_features.append(atom_extracter.encode(atom))

        pair_indices.append([atom.GetIdx(), atom.GetIdx()])
        bond_features.append(bond_extracter.encode(None))

        for neighbor in atom.GetNeighbors():
            bond = molecule.GetBondBetweenAtoms(atom.GetIdx(), neighbor.GetIdx())
            pair_indices.append([atom.GetIdx(), neighbor.GetIdx()])
            bond_features.append(bond_extracter.encode(bond))

    return np.array(atom_features), np.array(bond_features), np.array(pair_indices)


def get_graphs(smiles_list):

    atom_features_list = []
    bond_features_list = []
    pair_indices_list = []

    for smiles in smiles_list:
        molecule = molecule_creator(smiles)
        atom_features, bond_features, pair_indices = graph_creator(molecule)

        atom_features_list.append(atom_features)
        bond_features_list.append(bond_features)
        pair_indices_list.append(pair_indices)

    return (
        tf.ragged.constant(atom_features_list, dtype=tf.float32),
        tf.ragged.constant(bond_features_list, dtype=tf.float32),
        tf.ragged.constant(pair_indices_list, dtype=tf.int64),
    )


permuted_indices = np.random.permutation(np.arange(df.shape[0]))

# Подвыборка для тренировки: 80 % данных
train_index = permuted_indices[: int(df.shape[0] * 0.8)]
x_train = get_graphs(df.iloc[train_index].smiles)
y_train = df.iloc[train_index].exp

# Подвыборка для валидации: 20 % данных
valid_index = permuted_indices[int(df.shape[0] * 0.8) : int(df.shape[0])]
x_valid = get_graphs(df.iloc[valid_index].smiles)
y_valid = df.iloc[valid_index].exp

### Создание датасета

In [ ]:
def get_batch(x_batch, y_batch):

    atom_features, bond_features, pair_indices = x_batch

    num_atoms = atom_features.row_lengths()
    num_bonds = bond_features.row_lengths()

    molecule_indices = tf.range(len(num_atoms))
    molecule_indicator = tf.repeat(molecule_indices, num_atoms)

    gather_indices = tf.repeat(molecule_indices[:-1], num_bonds[1:])

    increment = tf.cumsum(num_atoms[:-1])
    increment = tf.pad(tf.gather(increment, gather_indices), [(num_bonds[0], 0)])

    pair_indices = pair_indices.merge_dims(outer_axis=0, inner_axis=1).to_tensor()
    pair_indices = pair_indices + increment[:, tf.newaxis]
    atom_features = atom_features.merge_dims(outer_axis=0, inner_axis=1).to_tensor()
    bond_features = bond_features.merge_dims(outer_axis=0, inner_axis=1).to_tensor()

    return (atom_features, bond_features, pair_indices, molecule_indicator), y_batch


def Dataset(X, y, batch_size=32, shuffle=False):
    dataset = tf.data.Dataset.from_tensor_slices((X, (y)))
    if shuffle:
        dataset = dataset.shuffle(1024)
    return dataset.batch(batch_size).map(get_batch, -1).prefetch(-1)


# Модель

## Механизм Message passing

In [ ]:
class NeighborsMessage(layers.Layer):
    def build(self, input_shape):
        self.atom_dim = input_shape[0][-1]
        self.bond_dim = input_shape[1][-1]
        self.kernel = self.add_weight(
            shape=(self.bond_dim, self.atom_dim * self.atom_dim),
            initializer="glorot_uniform",
            name="kernel",
        )
        self.bias = self.add_weight(
            shape=(self.atom_dim * self.atom_dim), initializer="zeros", name="bias", # создание bias вектора
        )
        self.built = True

    def call(self, inputs):
        atom_features, bond_features, pair_indices = inputs

        bond_features = tf.matmul(bond_features, self.kernel) + self.bias

        bond_features = tf.reshape(bond_features, (-1, self.atom_dim, self.atom_dim))

        neighbors_features = tf.gather(atom_features, pair_indices[:, 1])
        neighbors_features = tf.expand_dims(neighbors_features, axis=-1)

        weighted_features = tf.matmul(bond_features, neighbors_features)
        weighted_features = tf.squeeze(weighted_features, axis=-1)
        aggregated_features = tf.math.unsorted_segment_sum(
            weighted_features,
            pair_indices[:, 0],
            num_segments=tf.shape(atom_features)[0],
        )
        return aggregated_features


#
class MessagePassing(layers.Layer):
    def __init__(self, units, steps=4, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.steps = steps

    def build(self, input_shape):
        self.atom_dim = input_shape[0][-1]
        self.message_step = NeighborsMessage()
        self.pad_length = max(0, self.units - self.atom_dim)
        self.update_step = layers.GRUCell(self.atom_dim + self.pad_length)
        self.built = True

    def call(self, inputs):
        atom_features, bond_features, pair_indices = inputs

        atom_features_updated = tf.pad(atom_features, [(0, 0), (0, self.pad_length)])

        for i in range(self.steps):
            atom_features_aggregated = self.message_step(
                [atom_features_updated, bond_features, pair_indices]
            )

            atom_features_updated, _ = self.update_step(
                atom_features_aggregated, atom_features_updated
            )
        return atom_features_updated


## Обобщающая фукнция

In [ ]:

class AdderPaddind(layers.Layer):
    def __init__(self, batch_size, **kwargs):
        super().__init__(**kwargs)
        self.batch_size = batch_size

    def call(self, inputs):

        atom_features, molecule_indicator = inputs

        atom_features_partitioned = tf.dynamic_partition(
            atom_features, molecule_indicator, self.batch_size
        )

        num_atoms = [tf.shape(f)[0] for f in atom_features_partitioned]
        max_num_atoms = tf.reduce_max(num_atoms)
        atom_features_stacked = tf.stack(
            [
                tf.pad(f, [(0, max_num_atoms - n), (0, 0)])
                for f, n in zip(atom_features_partitioned, num_atoms)
            ],
            axis=0,
        )

        gather_indices = tf.where(tf.reduce_sum(atom_features_stacked, (1, 2)) != 0)
        gather_indices = tf.squeeze(gather_indices, axis=-1)
        return tf.gather(atom_features_stacked, gather_indices, axis=0)


class ReadoutWithAttention(layers.Layer):
    def __init__(
        self, num_heads=8, embed_dim=64, dense_dim=512, batch_size=32, **kwargs
    ):
        super().__init__(**kwargs)

        self.partition_padding = AdderPaddind(batch_size)
        self.attention = layers.MultiHeadAttention(num_heads, embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.average_pooling = layers.GlobalAveragePooling1D()

    def call(self, inputs):
        x = self.partition_padding(inputs)
        padding_mask = tf.reduce_any(tf.not_equal(x, 0.0), axis=-1)
        padding_mask = padding_mask[:, tf.newaxis, tf.newaxis, :]
        attention_output = self.attention(x, x, attention_mask=padding_mask)
        proj_input = self.layernorm_1(x + attention_output)
        proj_output = self.layernorm_2(proj_input + self.dense_proj(proj_input))
        return self.average_pooling(proj_output)


# Смешанная модель машинного обучения

In [ ]:

def GNNmodel(
    atom_dim,
    bond_dim,
    batch_size=32,
    message_units=64,
    message_steps=4,
    num_attention_heads=8,
    dense_units=256,
):

    atom_features = layers.Input((atom_dim), dtype="float32", name="atom_features")
    bond_features = layers.Input((bond_dim), dtype="float32", name="bond_features")
    pair_indices = layers.Input((2), dtype="int32", name="pair_indices")
    molecule_indicator = layers.Input((), dtype="int32", name="molecule_indicator")

    mp = MessagePassing(message_units, message_steps)(
        [atom_features, bond_features, pair_indices]
    )

    transf = ReadoutWithAttention(
        num_attention_heads, message_units, dense_units, batch_size
    )([mp, molecule_indicator])

    l1 = layers.Dense(dense_units, activation="relu")(transf)
    l2 = layers.Dense(1, activation="linear")(l1)

    full_model = keras.Model(
        inputs=[atom_features, bond_features, pair_indices, molecule_indicator],
        outputs=[l2]
    )
    clust_model = keras.Model(
        [atom_features, bond_features, pair_indices, molecule_indicator],
        l1
    )

    return full_model, clust_model

mpnn_full, mpnn_clust = GNNmodel(
    atom_dim=x_train[0][0][0].shape[0], bond_dim=x_train[1][0][0].shape[0],
)

mpnn_full.compile(
    loss=keras.losses.MeanSquaredError(),
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    metrics=[keras.metrics.MeanAbsoluteError(name="MAE")],
)

## Обучение

In [ ]:
train_dataset = Dataset(x_train, y_train)
valid_dataset = Dataset(x_valid, y_valid)

history = mpnn_full.fit(
    train_dataset,
    validation_data=valid_dataset,
    epochs=50,
    verbose=2
)

In [ ]:
min(history.history['val_MAE'])

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(history.history["MAE"], label="train MAE")
plt.plot(history.history["val_MAE"], label="valid MAE")
plt.xlabel("Эпохи", fontsize=16)
plt.ylabel("MAE", fontsize=16)
plt.legend(fontsize=16)

## Кластеризация

In [ ]:
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn import metrics
from scipy.spatial.distance import cdist

### Методы

In [ ]:
def cluster_sets(n, data_inp, data_targ, clust_index):
  clusters = []
  targets = []
  for i in range(n):
    clust = data_inp[clust_index == i]
    targ = data_targ[clust_index == i]
    clusters.append(np.array(clust))
    targets.append(np.array(targ))

  return np.array(clusters), np.array(targets)


In [ ]:
def VecDataset(X, y, batch_size=32, shuffle=False):
    dataset = tf.data.Dataset.from_tensor_slices((X, (y)))
    if shuffle:
        dataset = dataset.shuffle(1024)
    return dataset.batch(batch_size)

In [ ]:
def models_fiting(n, clusted_data_train, clusted_data_valid, clusted_target_train, clusted_target_valid):

  models = []

  for i in range(n):
    inp = layers.Input(256, dtype="float32")
    out = layers.Dense(1, activation="linear")(inp)
    new_model = keras.Model(inp, out)

    new_model.compile(
      loss=keras.losses.MeanSquaredError(),
      optimizer=keras.optimizers.Adam(learning_rate=1e-3),
      metrics=[keras.metrics.MeanAbsoluteError(name="MAE")],
    )

    train = VecDataset(clusted_data_train[i], clusted_target_train[i])
    valid = VecDataset(clusted_data_valid[i], clusted_target_valid[i])

    models.append(new_model.fit(
        train,
        validation_data=valid,
        epochs=50,
        verbose=0
    ))

  return models



In [ ]:
tr = mpnn_clust.predict(train_dataset)

In [ ]:
NUM_OF_CLUSTERS = 6

In [ ]:
knn = KMeans(n_clusters=NUM_OF_CLUSTERS).fit(tr)
hw = AgglomerativeClustering(n_clusters=NUM_OF_CLUSTERS, linkage='ward').fit(tr)
ha = AgglomerativeClustering(n_clusters=NUM_OF_CLUSTERS, linkage='average').fit(tr)
hc = AgglomerativeClustering(n_clusters=NUM_OF_CLUSTERS, linkage='complete').fit(tr)
hs = AgglomerativeClustering(n_clusters=NUM_OF_CLUSTERS, linkage='single').fit(tr)

In [ ]:
train_ds = mpnn_clust.predict(train_dataset)
valid_ds = mpnn_clust.predict(valid_dataset)

In [ ]:
clusted_train_ds, clusted_y_train = [], []
clusted_valid_ds, clusted_y_valid = [], []

KNN

In [ ]:
clusted_train_ds_buf, clusted_y_train_buf = cluster_sets(NUM_OF_CLUSTERS, train_ds, y_train, knn.predict(train_ds))
clusted_valid_ds_buf, clusted_y_valid_buf = cluster_sets(NUM_OF_CLUSTERS, valid_ds, y_valid, knn.predict(valid_ds))

clusted_train_ds.append(clusted_train_ds_buf)
clusted_y_train.append(clusted_y_train_buf)
clusted_valid_ds.append(clusted_valid_ds_buf)
clusted_y_valid.append(clusted_y_valid_buf)

Ward

In [ ]:
clusted_train_ds_buf, clusted_y_train_buf = cluster_sets(NUM_OF_CLUSTERS, train_ds, y_train, hw.fit_predict(train_ds))
clusted_valid_ds_buf, clusted_y_valid_buf = cluster_sets(NUM_OF_CLUSTERS, valid_ds, y_valid, hw.fit_predict(valid_ds))

clusted_train_ds.append(clusted_train_ds_buf)
clusted_y_train.append(clusted_y_train_buf)
clusted_valid_ds.append(clusted_valid_ds_buf)
clusted_y_valid.append(clusted_y_valid_buf)

Average

In [ ]:
clusted_train_ds_buf, clusted_y_train_buf = cluster_sets(NUM_OF_CLUSTERS, train_ds, y_train, ha.fit_predict(train_ds))
clusted_valid_ds_buf, clusted_y_valid_buf = cluster_sets(NUM_OF_CLUSTERS, valid_ds, y_valid, ha.fit_predict(valid_ds))

clusted_train_ds.append(clusted_train_ds_buf)
clusted_y_train.append(clusted_y_train_buf)
clusted_valid_ds.append(clusted_valid_ds_buf)
clusted_y_valid.append(clusted_y_valid_buf)

Complete

In [ ]:
clusted_train_ds_buf, clusted_y_train_buf = cluster_sets(NUM_OF_CLUSTERS, train_ds, y_train, hc.fit_predict(train_ds))
clusted_valid_ds_buf, clusted_y_valid_buf = cluster_sets(NUM_OF_CLUSTERS, valid_ds, y_valid, hc.fit_predict(valid_ds))

clusted_train_ds.append(clusted_train_ds_buf)
clusted_y_train.append(clusted_y_train_buf)
clusted_valid_ds.append(clusted_valid_ds_buf)
clusted_y_valid.append(clusted_y_valid_buf)

Single

In [ ]:
clusted_train_ds_buf, clusted_y_train_buf = cluster_sets(NUM_OF_CLUSTERS, train_ds, y_train, hs.fit_predict(train_ds))
clusted_valid_ds_buf, clusted_y_valid_buf = cluster_sets(NUM_OF_CLUSTERS, valid_ds, y_valid, hs.fit_predict(valid_ds))

clusted_train_ds.append(clusted_train_ds_buf)
clusted_y_train.append(clusted_y_train_buf)
clusted_valid_ds.append(clusted_valid_ds_buf)
clusted_y_valid.append(clusted_y_valid_buf)

In [ ]:
for clust_method in clusted_valid_ds:
  clusters_size = [len(cl) for cl in clust_method]
  print(clusters_size)

KNN

In [ ]:
mod_knn = models_fiting(NUM_OF_CLUSTERS, clusted_train_ds[0], clusted_valid_ds[0], clusted_y_train[0], clusted_y_valid[0])

Ward

In [ ]:
mod_hw = models_fiting(NUM_OF_CLUSTERS, clusted_train_ds[1], clusted_valid_ds[1], clusted_y_train[1], clusted_y_valid[1])

average

In [ ]:
mod_ha = models_fiting(NUM_OF_CLUSTERS, clusted_train_ds[2], clusted_valid_ds[2], clusted_y_train[2], clusted_y_valid[2])

complete

In [ ]:
mod_hc = models_fiting(NUM_OF_CLUSTERS, clusted_train_ds[3], clusted_valid_ds[3], clusted_y_train[3], clusted_y_valid[3])

single

In [ ]:
mod_hs = models_fiting(NUM_OF_CLUSTERS, clusted_train_ds[4], clusted_valid_ds[4], clusted_y_train[4], clusted_y_valid[4])

In [ ]:
print("KNN:")
for i in range(NUM_OF_CLUSTERS):
  print(min(mod_knn[i].history['val_MAE']))

print("HW:")
for i in range(NUM_OF_CLUSTERS):
  print(min(mod_hw[i].history['val_MAE']))

print("HA:")
for i in range(NUM_OF_CLUSTERS):
  print(min(mod_ha[i].history['val_MAE']))

print("HC:")
for i in range(NUM_OF_CLUSTERS):
  print(min(mod_hc[i].history['val_MAE']))

print("HS:")
for i in range(NUM_OF_CLUSTERS):
  print(min(mod_hs[i].history['val_MAE']))

## Метод локтя

In [ ]:
X = mpnn_clust.predict(train_dataset)

In [ ]:
distortions = []
inertias = []
mapping1 = {}
mapping2 = {}
K = range(1, 10)

for k in K:
	kmeanModel = KMeans(n_clusters=k).fit(X)
	kmeanModel.fit(X)

	distortions.append(sum(np.min(cdist(X, kmeanModel.cluster_centers_,
										'euclidean'), axis=1)) / X.shape[0])
	inertias.append(kmeanModel.inertia_)

	mapping1[k] = sum(np.min(cdist(X, kmeanModel.cluster_centers_,
								'euclidean'), axis=1)) / X.shape[0]
	mapping2[k] = kmeanModel.inertia_


In [ ]:
for key, val in mapping1.items():
	print(f'{key} : {val}')


In [ ]:
plt.plot(K, distortions, 'bx-')
plt.xlabel('Значение K')
plt.ylabel('Отклонение')
plt.show()